#### Prep

* Install yahoo_fin and requests_html
> Open terminal  then run:  pip install yahoo_fin requests_html

In [1]:
import h5py
import pandas as pd
import yahoo_fin as yf
from yahoo_fin.stock_info import *
import time
import shelve

In [2]:
# f = h5py.File('/data/stox.hdf5','w') 

cache = shelve.open('/data/stox.cache')


In [3]:
def get_nyse_df():
    df = pd.read_csv('/data/companylist.nyse.csv')
    df.columns = [c.strip().lower().replace(' ','_') for c in df.columns]
    df['marketcap'] = df['marketcap'].str.replace(' ','').str.replace(',','').str.replace('-','0').astype(float)
    for c in df.columns:
        if df[c].dtype == object: 
            df[c] = df[c].astype(str)
    return df

def get_nasdaq_df():
    df = pd.read_csv('/data/companylist.nasdaq.csv')

    df.columns = [c.strip().lower().replace(' ','_') for c in df.columns]

    # df['marketcap'] = df['marketcap']\
    #     .str.replace('$','').str.replace(' ','').str.replace('n/a','0').str.replace(',','')\
    #     .str.replace('-','0')
    def ston(x):
        v = x
        try:
            v = str(x).strip().replace('$','').replace(',','').replace('n/a','0.0')
            v = float(v[:-1])*1e9 if v[-1] == 'B' else float(v[:-1])*1e6
            return v
        except:
            return 0
    df['marketcap'] = df['marketcap'].apply(ston)
    return df


In [4]:
# df = pd.read_csv('/data/companylist.nasdaq.csv')

# df.columns = [c.strip().lower().replace(' ','_') for c in df.columns]

# # df['marketcap'] = df['marketcap']\
# #     .str.replace('$','').str.replace(' ','').str.replace('n/a','0').str.replace(',','')\
# #     .str.replace('-','0')
# def ston(x):
#     v = x
#     try:
#         v = str(x).strip().replace('$','').replace(',','').replace('n/a','0.0')
#         v = float(v[:-1])*1e9 if v[-1] == 'B' else float(v[:-1])*1e6
#         return v
#     except:
#         return 0
# df['marketcap'] = df['marketcap'].apply(ston)
# # for c in df.columns:
# #     if df[c].dtype == object: 
# #         df[c] = df[c].astype(str)

def filter_mid_cap_5y(df):
    mid_cap = df[ (5e9 < df.marketcap) &  (df.marketcap < 11e10) 
                 & (df.ipoyear > 2014) & (df.ipoyear <= 2019)]
    return mid_cap

In [5]:
df = get_nasdaq_df()
mid_cap_5y_nasdaq = filter_mid_cap_5y(df)

df = get_nyse_df()
mid_cap_5y_nyse = filter_mid_cap_5y(df)

#                 & ((df.sector == 'Health Care') | (df.sector == 'Technology'))]
# mid_cap_10y.to_hdf('/data/stox.hdf5', key='mid_cap_10y', mode='w')

top_nasdaq = mid_cap_5y_nasdaq.sort_values(by=['lastsale'],ascending=False).head(30).symbol.to_list()
top_nyse = mid_cap_5y_nyse.sort_values(by=['lastsale'],ascending=False).head(30).symbol.to_list()

top_stox = sorted(list(set(top_nasdaq+top_nyse)))


In [6]:
# top

In [7]:
df = get_nyse_df()
mid_cap_10y1 = df[ (5e9 < df.marketcap) &  (df.marketcap < 11e10) &  (df.ipoyear > 2014) & (df.ipoyear < 2018)]
print (mid_cap_10y1)


my_stox = "SBUX,MSFT,AMZN,GOOG,C,FB,TWTR,WMT".split(',')


     symbol                                              name  lastsale  \
72      AQN                 Algonquin Power & Utilities Corp.     12.98   
104     AYX                                     Alteryx, Inc.    135.33   
105    ATUS                                  Altice USA, Inc.     26.70   
262     ATH                               Athene Holding Ltd.     39.20   
275     AGR                                    Avangrid, Inc.     49.36   
303    BHGE                        Baker Hughes, a GE company     23.57   
384     BKI                                Black Knight, Inc.     61.38   
531    CABO                                   Cable One, Inc.   1261.41   
586    CVNA                                       Carvana Co.     80.43   
616     EBR  Centrais Electricas Brasileiras S.A.- Eletrobras     10.90   
720    CCEP                   Coca-Cola European Partners plc     54.36   
947      DD                           DuPont de Nemours, Inc.     68.48   
1031   ENIC              

In [8]:
def gest(s):
    ckey = f"perf_est_{s.lower()}"
    if ckey in cache:
        return cache[ckey]
    time.sleep(2)
    try:
        ai = get_analysts_info(s)
        v = ai['Growth Estimates'] if ai else None
        cache[ckey] = v
        return v
    except:
        cache[ckey] = None
        return None

In [9]:
def evaluate_stox(stox):
    values = [(s,gest(s)) for s in stox]
    values = [(s,v) for s,v in values if v is not None]
    # values.sort(key=lambda sv: sv[1])
    # for s in top:
    #     v = gest(s)
    #     if v is not None:
    #         print(f'-------- {s:>8} {v.iloc[4,1]} {v.iloc[3,1]} {v.iloc[2,1]} {v.iloc[5,1]}   -------')
    #     else:
    #         print(f'-------- {s} None  -------')
    for s,v in values:
        print(f'-------- {s:>8} {v.iloc[4,1]} {v.iloc[3,1]} {v.iloc[2,1]} {v.iloc[5,1]}   -------')


In [10]:
evaluate_stox(my_stox)
evaluate_stox(top_stox)


--------     SBUX 10.55% 12.50% 7.80% 14.45%   -------
--------     MSFT 14.50% 12.40% 13.70% 18.17%   -------
--------     AMZN 28.00% 29.80% 2.60% 100.60%   -------
--------     GOOG 15.82% 17.20% 6.10% nan   -------
--------        C 13.00% 9.60% 6.10% 26.09%   -------
--------       FB 11.46% 43.10% -15.30% 42.12%   -------
--------     TWTR 13.90% -62.80% 177.90% 51.68%   -------
--------      WMT 5.18% 4.40% 1.60% 0.53%   -------
--------     ADPT nan 23.60% nan nan   -------
--------      ALC 3.42% 5.80% nan nan   -------
--------     ARGX nan -87.40% -37.40% nan   -------
--------     ASND 24.00% -11.00% -28.70% nan   -------
--------     AVLR -53.90% 7.10% 79.10% nan   -------
--------      AYX 39.80% 43.10% 3,000.00% nan   -------
--------     BGNE 24.00% -15.40% -10.00% nan   -------
--------      BKI 8.69% 5.70% 3.20% 21.29%   -------
--------     BYND -226.30% 327.80% nan nan   -------
--------     CABO 15.50% 37.40% 8.70% 20.89%   -------
--------     CCEP 10.14% 9.30% 10

In [11]:
gest('MSFT')

,Growth Estimates,MSFT,Industry,Sector,S&P 500
0,Current Qtr.,20.00%,NaN,NaN,0.01
1,Next Qtr.,8.80%,NaN,NaN,0.06
2,Current Year,13.70%,NaN,NaN,-0.02
3,Next Year,12.40%,NaN,NaN,0.14
4,Next 5 Years (per annum),14.50%,NaN,NaN,0.07
5,Past 5 Years (per annum),18.17%,NaN,NaN,NaN


In [12]:
bsmsft,bsaapl = get_balance_sheet('MSFT'),get_balance_sheet('AAPL')

In [13]:
bsmsft

,Breakdown,6/29/2019,6/29/2018,6/29/2017,6/29/2016
0,Current Assets,None,None,None,None
1,Cash,None,None,None,None
2,Cash And Cash Equivalents,11356000,11946000,7663000,6510000
3,Short Term Investments,122463000,121822000,125318000,106730000
4,Total Cash,133819000,133768000,132981000,113240000
5,Net Receivables,29524000,26481000,19792000,18277000
6,Inventory,2063000,2662000,2181000,2251000
7,Other Current Assets,10146000,6751000,4897000,5892000
8,Total Current Assets,175552000,169662000,159851000,139660000
9,Non-current assets,None,None,None,None
